In [ ]:
# === ended with an optimal accuracy of ~33%, did not include as final model ===

In [1]:
# installing tensorflow (to make it work in jupyter notebook)
!pip install tensorflow

In [2]:
# import dependencies
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np

# import movie csv
movies_df = pd.read_csv('Resources/movies_cleaned.csv')

# display df
movies_df.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [3]:
# create new revenue column
movies_df['percent_revenue'] = (movies_df['gross']/ movies_df['budget'])*100

# include only necessary columns
cleaned_movies_df = movies_df[['rating', 'genre', 'score', 'votes', 'director',
                               'writer', 'star', 'budget', 'gross', 'percent_revenue',
                               'company','runtime']]

# display newly cleaned df
cleaned_movies_df.head()

,rating,genre,score,votes,director,writer,star,budget,gross,percent_revenue,company,runtime
0,R,Drama,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,19000000.0,46998772.0,247.361958,Warner Bros.,146.0
1,R,Adventure,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,4500000.0,58853106.0,1307.846800,Columbia Pictures,104.0
2,PG,Action,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,18000000.0,538375067.0,2990.972594,Lucasfilm,124.0
3,PG,Comedy,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,3500000.0,83453539.0,2384.386829,Paramount Pictures,88.0
4,R,Comedy,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,6000000.0,39846344.0,664.105733,Orion Pictures,98.0


In [4]:
# display columns types
cleaned_movies_df.dtypes

rating              object
genre               object
score              float64
votes              float64
director            object
writer              object
star                object
budget             float64
gross              float64
percent_revenue    float64
company             object
runtime            float64
dtype: object

In [5]:
# encode non-numerical data
dummies = pd.get_dummies(data=cleaned_movies_df)

# combining the newly encoded data with the original dataframe
concat_df = pd.concat([cleaned_movies_df, dummies], axis=1)

# removing duplicate columns
concat_df = concat_df.loc[:,~concat_df.columns.duplicated()].copy()

#dropping the old, not encoded columns
concat_df = concat_df.drop(columns=['rating','genre', 'director', 'writer', 'star', 'company'])

concat_df.head()

,score,votes,budget,gross,percent_revenue,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,...,company_Zentropa Entertainments,company_Zeta Entertainment,company_Zoetrope Studios,company_Zucker Brothers Productions,company_double A Films,company_erbp,company_i am OTHER,company_i5 Films,company_micro_scope,company_thefyzz
0,8.4,927000.0,19000000.0,46998772.0,247.361958,146.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5.8,65000.0,4500000.0,58853106.0,1307.846800,104.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,8.7,1200000.0,18000000.0,538375067.0,2990.972594,124.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,7.7,221000.0,3500000.0,83453539.0,2384.386829,88.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,7.3,108000.0,6000000.0,39846344.0,664.105733,98.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
# create bins for percentages
bins = [0, 100, 200, 300]

labels = np.array([1, 2, 3, 4])

concat_df['profit_category'] = labels[np.searchsorted(bins, concat_df['percent_revenue'])-1]

concat_df.head()

,score,votes,budget,gross,percent_revenue,runtime,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,...,company_Zeta Entertainment,company_Zoetrope Studios,company_Zucker Brothers Productions,company_double A Films,company_erbp,company_i am OTHER,company_i5 Films,company_micro_scope,company_thefyzz,profit_category
0,8.4,927000.0,19000000.0,46998772.0,247.361958,146.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
1,5.8,65000.0,4500000.0,58853106.0,1307.846800,104.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4
2,8.7,1200000.0,18000000.0,538375067.0,2990.972594,124.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4
3,7.7,221000.0,3500000.0,83453539.0,2384.386829,88.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4
4,7.3,108000.0,6000000.0,39846344.0,664.105733,98.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4


In [7]:
# split our preprocessed data into our features and target arrays
y = concat_df['profit_category']
X = concat_df.drop(columns='profit_category')

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# define the model
model = tf.keras.models.Sequential()

# first hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=8619))

# second hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="relu"))

# output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               862000    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 872201 (3.33 MB)
Trainable params: 872201 (3.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
fit_model = model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
128/128 [==============================] - 3s 13ms/step - loss: -3441130752.0000 - accuracy: 0.3188
Epoch 2/100
128/128 [==============================] - 2s 13ms/step - loss: -3519680256.0000 - accuracy: 0.3188
Epoch 3/100
128/128 [==============================] - 2s 15ms/step - loss: -3599083008.0000 - accuracy: 0.3188
Epoch 4/100
128/128 [==============================] - 2s 15ms/step - loss: -3680216320.0000 - accuracy: 0.3188
Epoch 5/100
128/128 [==============================] - 2s 18ms/step - loss: -3764415744.0000 - accuracy: 0.3188
Epoch 6/100
128/128 [==============================] - 2s 15ms/step - loss: -3848289792.0000 - accuracy: 0.3188
Epoch 7/100
128/128 [==============================] - 2s 13ms/step - loss: -3933821440.0000 - accuracy: 0.3188
Epoch 8/100
128/128 [==============================] - 2s 12ms/step - loss: -4021837568.0000 - accuracy: 0.3188
Epoch 9/100
128/128 [==============================] - 2s 12ms/step - loss: -4111024128.0000 - accuracy:

128/128 [==============================] - 2s 13ms/step - loss: -13444017152.0000 - accuracy: 0.3188
Epoch 74/100
128/128 [==============================] - 2s 12ms/step - loss: -13652206592.0000 - accuracy: 0.3188
Epoch 75/100
128/128 [==============================] - 2s 13ms/step - loss: -13857886208.0000 - accuracy: 0.3188
Epoch 76/100
128/128 [==============================] - 2s 12ms/step - loss: -14069168128.0000 - accuracy: 0.3188
Epoch 77/100
128/128 [==============================] - 2s 12ms/step - loss: -14278307840.0000 - accuracy: 0.3188
Epoch 78/100
128/128 [==============================] - 2s 13ms/step - loss: -14489143296.0000 - accuracy: 0.3188
Epoch 79/100
128/128 [==============================] - 2s 12ms/step - loss: -14702662656.0000 - accuracy: 0.3188
Epoch 80/100
128/128 [==============================] - 2s 12ms/step - loss: -14918286336.0000 - accuracy: 0.3188
Epoch 81/100
128/128 [==============================] - 2s 12ms/step - loss: -15140600832.0000 - accu

In [21]:
# create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()

    # allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=8619))

    # allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return model

In [22]:
# import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from .\untitled_project\tuner0.json


In [23]:
# run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 00m 11s]
val_accuracy: 0.3303834795951843

Best val_accuracy So Far: 0.3303834795951843
Total elapsed time: 00h 14m 44s


In [25]:
# get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 1,
 'units_0': 7,
 'units_1': 3,
 'units_2': 5,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [26]:
# evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

43/43 - 0s - loss: -5.1064e+00 - accuracy: 0.3304 - 286ms/epoch - 7ms/step
Loss: -5.1063714027404785, Accuracy: 0.3303834795951843
